In [42]:
%matplotlib inline

"""This version of the file is just for cooking up new ideas. gan_skip.ipynb is the master file."""

import os
# running with non gpu singularity container, so commented out the next line to use CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
tf.set_random_seed(42)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
print "import tensorflow"
           
import keras.backend.tensorflow_backend as K

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Lambda
from keras.layers import Input, merge, Concatenate, concatenate, Add
from keras.losses import binary_crossentropy
from keras.utils import plot_model
print "import keras"

import numpy as np
#from tqdm import tqdm
import time
import pickle
import sys

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

print "import matplotlib"

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import binned_statistic_2d

print "import sklearn"

np.random.seed(42)


import tensorflow
import keras
import matplotlib
import sklearn


In [3]:
def Minv(cols,ptetaphi=False,nopy2=False):
    """
    Computes M for two objects given the cartesian momentum projections
    if `ptetaphi` is True, then assumes the 8 input columns are cylindrical eptetaphi
    if `nopy2` is True, input is 7 columns with no py2
    """
    if ptetaphi:
        cols = ptetaphi_to_cartesian(cols)
    if nopy2:
        M2 = (cols[:,0]+cols[:,4])**2
        M2 -= (cols[:,1]+cols[:,5])**2
        M2 -= (cols[:,2]          )**2
        M2 -= (cols[:,3]+cols[:,6])**2
    else:
        M2 = (cols[:,0]+cols[:,4])**2
        M2 -= (cols[:,1]+cols[:,5])**2
        M2 -= (cols[:,2]+cols[:,6])**2
        M2 -= (cols[:,3]+cols[:,7])**2
    return np.sqrt(M2)

def cartesian_to_ptetaphi(eight_cartesian_cols):
    """
    Takes 8 columns as cartesian e px py pz e px py pz
    and converts to e pt eta phi e pt eta phi
    """
    e1 =  eight_cartesian_cols[:,0]
    e2 =  eight_cartesian_cols[:,4]
    px1 = eight_cartesian_cols[:,1]
    px2 = eight_cartesian_cols[:,5]
    py1 = eight_cartesian_cols[:,2]
    py2 = eight_cartesian_cols[:,6]
    pz1 = eight_cartesian_cols[:,3]
    pz2 = eight_cartesian_cols[:,7]
    p1 = np.sqrt(px1**2+py1**2+pz1**2)
    p2 = np.sqrt(px2**2+py2**2+pz2**2)
    pt1 = np.sqrt(px1**2+py1**2)
    pt2 = np.sqrt(px2**2+py2**2)
    phi1 = np.arctan2(py1,px1)
    phi2 = np.arctan2(py2,px2)
    eta1 = np.arctanh(pz1/p1)
    eta2 = np.arctanh(pz2/p2)
    return np.c_[e1,pt1,eta1,phi1,e2,pt2,eta2,phi2]

def ptetaphi_to_cartesian(eight_eptetaphi_cols):
    """
    Takes 8 columns as e pt eta phi e pt eta phi
    and converts to e px py pz e px py pz
    """
    e1 =  eight_eptetaphi_cols[:,0]
    e2 =  eight_eptetaphi_cols[:,4]
    pt1 =  eight_eptetaphi_cols[:,1]
    pt2 =  eight_eptetaphi_cols[:,5]
    eta1 =  eight_eptetaphi_cols[:,2]
    eta2 =  eight_eptetaphi_cols[:,6]
    phi1 =  eight_eptetaphi_cols[:,3]
    phi2 =  eight_eptetaphi_cols[:,7]
    px1 = np.abs(pt1)*np.cos(phi1)
    px2 = np.abs(pt2)*np.cos(phi2)
    py1 = np.abs(pt1)*np.sin(phi1)
    py2 = np.abs(pt2)*np.sin(phi2)
    pz1 = np.abs(pt1)/np.tan(2.0*np.arctan(np.exp(-1.*eta1)))
    pz2 = np.abs(pt2)/np.tan(2.0*np.arctan(np.exp(-1.*eta2)))
    return np.c_[e1,px1,py1,pz1,e2,px2,py2,pz2]

def get_dphi(px1,py1,px2,py2):
    phi1 = np.arctan2(py1,px1)
    phi2 = np.arctan2(py2,px2)
    dphi = phi1-phi2
    dphi[dphi>np.pi] -= 2*np.pi
    dphi[dphi<-np.pi] += 2*np.pi 
    return dphi

def get_rotated_pxpy(px1,py1,px2,py2):
    """
    rotates two leptons such that phi2 = 0
    """
    pt1 = np.sqrt(px1**2+py1**2)
    pt2 = np.sqrt(px2**2+py2**2)
    phi1 = np.arctan2(py1,px1)
    phi2 = np.arctan2(py2,px2)
    px1p = pt1*np.cos(phi1-phi2)
    py1p = pt1*np.sin(phi1-phi2)
    px2p = pt2*np.cos(phi2-phi2)
    return px1p,py1p,px2p,np.zeros(len(pt2))
    
def cartesian_zerophi2(coords,ptetaphi=False):
    """
    returns 8-1=7 columns rotating leptons such that phi2 is 0 (and removing it)
    if `ptetaphi` is True, then return eptetaphi instead of epxpypz
    """
    lepcoords_cyl = cartesian_to_ptetaphi(coords)
    phi1 = lepcoords_cyl[:,3]
    phi2 = lepcoords_cyl[:,7]
    dphi = phi1-phi2
    dphi[dphi>np.pi] -= 2*np.pi
    dphi[dphi<-np.pi] += 2*np.pi
    lepcoords_cyl[:,3] = dphi
    lepcoords_cyl[:,7] = 0.
    if ptetaphi:
        return np.delete(lepcoords_cyl, [7], axis=1)
    else:
        return np.delete(ptetaphi_to_cartesian(lepcoords_cyl), [6], axis=1)

In [24]:
def invmass_from_8cartesian(x):
    invmass = K.sqrt(
                (x[:,0:1]+x[:,4:5])**2-
                (x[:,1:2]+x[:,5:6])**2-
                (x[:,2:3]+x[:,6:7])**2-
                (x[:,3:4]+x[:,7:8])**2
                )
    return invmass

def invmass_from_8cartesian_nopy2(x):
    invmass = K.sqrt(
                (x[:,0:1]+x[:,4:5])**2-
                (x[:,1:2]+x[:,5:6])**2-
                (x[:,2:3]         )**2-
                (x[:,3:4]+x[:,6:7])**2
                )
    return invmass

def get_first_N(x,N=19):
    return x[:,0:N]

def add_invmass_from_8cartesian(x):
    return K.concatenate([x,invmass_from_8cartesian(x)])


def fix_outputs(x):
    """
    Take nominal delphes format of 19 columns and fix some columns
    """
    return K.concatenate([
        # x[:,0:21],
        x[:,0:7], # epxpypz for lep1,lep2 -1 for no py2
        x[:,7:8], # nvtx
        K.sign(x[:,8:10]), # q1 q2
        x[:,10:12], # iso1 iso2
        x[:,12:14], # met, metphi
        x[:,14:19], # jet pts
        ])

def custom_loss(c, loss_type = "force_mll"):
    if loss_type == "force_mll":
        def loss_func(y_true, y_pred_mll):
            y_true = y_true[:,0]
            y_pred = y_pred_mll[:,0]
            mll_pred = y_pred_mll[:,1]

            mll_loss = K.mean(K.abs(mll_pred - 91.2))

    #         pseudomll = K.random_normal_variable(shape=(1,1), mean=91.2, scale=2)
    #         mll_loss = K.mean((mll_pred - pseudomll)**2)

            return binary_crossentropy(y_true, y_pred) + c*mll_loss
        return loss_func
    elif loss_type == "force_z_width":
        def loss_func(y_true, y_pred_mll):
            y_true = y_true[:,0]
            y_pred = y_pred_mll[:,0]
            mll_pred = y_pred_mll[:,1]
            
            mll_loss = K.mean(K.abs(mll_pred - 91.2))
            mll_sigma_loss = K.abs(K.std(mll_pred) - 7.67)

            return binary_crossentropy(y_true, y_pred) + c*mll_loss + c*mll_sigma_loss
        return loss_func
        
    else:
        raise ValueError("Can not make loss function of type %s" % loss_type)

In [63]:
def METPhiMap(metphis):
    """Works so long as the tails are constrained within [-2pi, 2pi], maps everything from [-pi,pi]"""
    return ((metphis+np.pi) % (2*np.pi)) - np.pi

def make_plots_new(preds,reals,title="",fname="",show_pred=True,wspace=0.1,hspace=0.3,tightlayout=True,visible=False):
    nrows, ncols = 5,5
    fig, axs = plt.subplots(nrows,ncols,figsize=(16,13))
#     fig, axs = plt.subplots(nrows,ncols,figsize=(12,10))
#     fig.subplots_adjust(wspace=0.1,hspace=0.3)
    fig.subplots_adjust(wspace=wspace,hspace=hspace)


    info = [
        ["mll",(60,120,50)],
        ["lep1_e",(0,250,50)],
        ["lep1_px",(-100,100,50)],
        ["lep1_py",(-100,100,50)],
        ["lep1_pz",(-200,200,50)],
        ["lep2_e",(0,250,50)],
        ["lep2_px",(-100,100,50)],
        ["lep2_pz",(-200,200,50)],
        ["dphi",(-4,4,50)],
        ["nvtxs",(0,50,350)],
        ["met",(0,150,50)],
        ["metphi",(-6,6,50)],
        ["lep1_charge",(-7,7,30)],
        ["lep2_charge",(-7,7,30)],
        ["lep1_iso",(0,2.0,30)],
        ["lep2_iso",(0,2.0,30)],
        ["jet_pt1",(0,100,50)],
        ["jet_pt2",(0,100,50)],
        ["jet_pt3",(0,100,50)],
        ["jet_pt4",(0,100,50)],
        ["jet_pt5",(0,100,50)],
        ["njets",(0,7,7)],

    ]
    for axx in axs:
        for ax in axx:
            ax.get_yaxis().set_visible(False)
    for ic,(cname,crange) in enumerate(info):
        if cname == "mll":
            real = reals["mll"]
            pred = Minv(preds,ptetaphi=False,nopy2=True)
        elif cname == "lep1_e": real, pred = reals[cname], preds[:,0]
        elif cname == "lep1_pz": real, pred = reals[cname], preds[:,3]
        elif cname == "lep2_e": real, pred = reals[cname], preds[:,4]
        elif cname == "lep2_pz": real, pred = reals[cname], preds[:,6]
        elif cname == "lep1_px": 
            real = reals[cname]
            pred = preds[:,1]
        elif cname == "lep1_py":
            real = reals[cname]
            pred = preds[:,2]
        elif cname == "lep2_px":
            real = reals[cname]
            pred = preds[:,5]
        elif cname == "dphi":
            real = get_dphi(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], np.zeros(len(reals)))
            pred = get_dphi(preds[:,1], preds[:,2], preds[:,5], np.zeros(len(preds)))
        elif cname == "nvtxs": real, pred = reals[cname], np.round(preds[:,7])
        elif cname == "lep1_charge": real, pred = reals[cname], preds[:,8]
        elif cname == "lep2_charge": real, pred = reals[cname], preds[:,9]
        elif cname == "lep1_iso": real, pred = reals[cname], preds[:,10]
        elif cname == "lep2_iso": real, pred = reals[cname], preds[:,11]
        elif cname == "met": real, pred = reals[cname], preds[:,12]
        elif cname == "metphi": real, pred = reals[cname], preds[:,13]
        elif cname == "jet_pt1": real, pred = reals[cname], preds[:,14]
        elif cname == "jet_pt2": real, pred = reals[cname], preds[:,15]
        elif cname == "jet_pt3": real, pred = reals[cname], preds[:,16]
        elif cname == "jet_pt4": real, pred = reals[cname], preds[:,17]
        elif cname == "jet_pt5": real, pred = reals[cname], preds[:,18]
        elif cname == "njets":
            real = \
                1*(reals["jet_pt1"] > 10) + \
                1*(reals["jet_pt2"] > 10) + \
                1*(reals["jet_pt3"] > 10) + \
                1*(reals["jet_pt4"] > 10) + \
                1*(reals["jet_pt5"] > 10)
            pred = \
                1*(preds[:,14] > 10) + \
                1*(preds[:,15] > 10) + \
                1*(preds[:,16] > 10) + \
                1*(preds[:,17] > 10) + \
                1*(preds[:,18] > 10)
        idx = ic // ncols, ic % ncols
        bins_real = axs[idx].hist(real, range=crange[:2],bins=crange[-1], histtype="step", lw=1.5,density=True)
        if show_pred:
            bins_pred = axs[idx].hist(pred, range=crange[:2],bins=crange[-1], histtype="step", lw=1.5,density=True)
        axs[idx].set_xlabel("{}".format(cname))
        axs[idx].get_yaxis().set_visible(False)
    #     axs[idx].set_yscale("log", nonposy='clip')
    _ = axs[0,0].legend(["True","Pred"], loc='upper right')
    _ = axs[0,0].set_title(title)
    if tightlayout:
        plt.tight_layout()
    if fname:
        fig.savefig(fname)
    if not visible:
        plt.close(fig)

def make_plots_old(preds,reals,title="",fname="", visible=False):
    nrows, ncols = 5,5
    fig, axs = plt.subplots(nrows,ncols,figsize=(16,13))
    fig.subplots_adjust(wspace=0.1,hspace=0.3)


    #print(preds)
    info = [
        ["mll",(60,120,50)],
        ["lep1_e",(0,250,50)],
        ["lep1_px",(-100,100,50)],
        ["lep1_py",(-100,100,50)],
        ["lep1_pz",(-200,200,50)],
        ["lep2_e",(0,250,50)],
        ["lep2_px",(-100,100,50)],
        ["lep2_pz",(-200,200,50)],
        ["dphi",(-4,4,50)],
        ["nvtxs",(0,50,350)],
        ["met",(0,150,50)],
        ["metphi",(-6,6,50)],
        ["lep1_charge",(-7,7,30)],
        ["lep2_charge",(-7,7,30)],
        ["lep1_iso",(0,0.2,30)],
        ["lep2_iso",(0,0.2,30)],
        ["genjet_pt1",(0,100,50)],
        ["genjet_pt2",(0,100,50)],
        ["genjet_pt3",(0,100,50)],
        ["genjet_pt4",(0,100,50)],
        ["genjet_pt5",(0,100,50)],

    ]
    for ic,(cname,crange) in enumerate(info):
        if cname == "mll":
            real = reals["mll"]
            pred = Minv(preds,ptetaphi=False,nopy2=True)
        elif cname == "lep1_e": real, pred = reals[cname], preds[:,0]
        elif cname == "lep1_pz": real, pred = reals[cname], preds[:,3]
        elif cname == "lep2_e": real, pred = reals[cname], preds[:,4]
        elif cname == "lep2_pz": real, pred = reals[cname], preds[:,6]
        elif cname == "lep1_px": 
            real = get_rotated_pxpy(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], reals["lep2_py"])[0]
            pred = preds[:,1]
        elif cname == "lep1_py":
            real = get_rotated_pxpy(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], reals["lep2_py"])[1]
            pred = preds[:,2]
        elif cname == "lep2_px":
            real = get_rotated_pxpy(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], reals["lep2_py"])[2]
            pred = preds[:,5]
        elif cname == "dphi":
            real = get_dphi(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], reals["lep2_py"])
            pred = get_dphi(preds[:,1], preds[:,2], preds[:,5], np.zeros(len(preds)))
        elif cname == "nvtxs": real, pred = reals[cname], np.round(preds[:,7])
        elif cname == "lep1_charge": real, pred = reals[cname], preds[:,8]
        elif cname == "lep2_charge": real, pred = reals[cname], preds[:,9]
        elif cname == "lep1_iso": real, pred = reals[cname], preds[:,10]
        elif cname == "lep2_iso": real, pred = reals[cname], preds[:,11]
        elif cname == "met": real, pred = reals[cname], preds[:,12]
        elif cname == "metphi": real, pred = reals[cname], METPhiMap(preds[:,13])
        elif cname == "genjet_pt1": real, pred = reals[cname], preds[:,14]
        elif cname == "genjet_pt2": real, pred = reals[cname], preds[:,15]
        elif cname == "genjet_pt3": real, pred = reals[cname], preds[:,16]
        elif cname == "genjet_pt4": real, pred = reals[cname], preds[:,17]
        elif cname == "genjet_pt5": real, pred = reals[cname], preds[:,18]
        idx = ic // ncols, ic % ncols
        bins_real = axs[idx].hist(real, range=crange[:2],bins=crange[-1], histtype="step", lw=2,density=True)
        bins_pred = axs[idx].hist(pred, range=crange[:2],bins=crange[-1], histtype="step", lw=2,density=True)
        axs[idx].set_xlabel("{}".format(cname))
        axs[idx].get_yaxis().set_visible(False)
    #     axs[idx].set_yscale("log", nonposy='clip')
    _ = axs[0,0].legend(["True","Pred"], loc='upper right')
    _ = axs[0,0].set_title(title)
    plt.tight_layout()
    if fname:
        fig.savefig(fname)
    if not visible:
        plt.close(fig)

make_plots = None #will be set by get_noise method

In [64]:
def load_data(input_file):
    data = np.load(input_file)
    return data

In [65]:
def get_noise(input_file, data, noise_type, noise_shape, amount=1024, use_ptetaphi_additionally=False, use_delphes=True):
    # nominal
    global make_plots
    
    if "data_Nov10.npa" in input_file:
        make_plots = make_plots_old
        
        lepcoords = np.c_[
            data["lep1_e"],
            data["lep1_px"],
            data["lep1_py"],
            data["lep1_pz"],
            data["lep2_e"],
            data["lep2_px"],
            data["lep2_py"],
            data["lep2_pz"],
        ]
        lepcoords_dphi = cartesian_zerophi2(lepcoords)

        nvtx_smeared = np.round(np.random.normal(data["nvtxs"],0.5))
        X_train = np.c_[
            lepcoords_dphi, # 7 columns
            nvtx_smeared, # 1 column
            data["lep1_charge"], data["lep2_charge"],
            data["lep1_iso"], data["lep2_iso"],
            data["met"], data["metphi"],
            data["genjet_pt1"],
            data["genjet_pt2"],
            data["genjet_pt3"],
            data["genjet_pt4"],
            data["genjet_pt5"],
        ].astype(np.float32)

    elif "total_Zmumu_13TeV_PU20_v2.npa" in input_file:
        make_plots = make_plots_new
        lepcoords = np.c_[
            data["lep1_e"],
            data["lep1_px"],
            data["lep1_py"],
            data["lep1_pz"],
            data["lep2_e"],
            data["lep2_px"],
#                 data["lep2_py"],
            data["lep2_pz"],
        ]
#             lepcoords_dphi = cartesian_zerophi2(lepcoords)

        nvtx_smeared = np.round(np.random.normal(data["nvtxs"],0.5))
        X_train = np.c_[
#                 lepcoords_dphi, # 7 columns
            lepcoords, # 7 columns
            nvtx_smeared, # 1 column
            data["lep1_charge"], data["lep2_charge"],
            data["lep1_iso"], data["lep2_iso"],
            data["met"], data["metphi"],
            data["jet_pt1"],
            data["jet_pt2"],
            data["jet_pt3"],
            data["jet_pt4"],
            data["jet_pt5"],
        ].astype(np.float32)
    else:
        make_plots = make_plots_old
        X_train = data[:,range(1,1+8)]
        if use_ptetaphi_additionally:
            X_train = np.c_[X_train, cartesian_to_ptetaphi(X_train)]

            
    if noise_type == 1:
        noise_half = np.random.normal(0, 1, (amount//2, noise_shape[0]))
        noise_full = np.random.normal(0, 1, (amount, noise_shape[0]))

    elif noise_type == 2: # random soup, 4,2,2 have to be modified to sum to noise_shape[0]
        ngaus = noise_shape[0] // 2
        nflat = (noise_shape[0] - ngaus) // 2
        nexpo = noise_shape[0] - nflat - ngaus
        noise_gaus = np.random.normal( 0, 1, (amount//2+amount, ngaus))
        noise_flat = np.random.uniform(-1, 1, (amount//2+amount, nflat))
        noise_expo = np.random.exponential( 1,    (amount//2+amount, nexpo))
        noise = np.c_[ noise_gaus,noise_flat,noise_expo ]
        noise_half = noise[:amount//2]
        noise_full = noise[-amount:]

    elif noise_type == 3: # truth conditioned

#             noise_half = np.c_[ 
#                     self.X_train[np.random.randint(0, self.X_train.shape[0], amount//2)], 
#                     np.random.normal(0, 1, (amount//2,self.noise_shape[0]-self.X_train.shape[1]))
#                     ]
#             noise_full = np.c_[ 
#                     self.X_train[np.random.randint(0, self.X_train.shape[0], amount)], 
#                     np.random.normal(0, 1, (amount,self.noise_shape[0]-self.X_train.shape[1]))
#                     ]

        npurenoise = noise_shape[0]-X_train.shape[1]
        ngaus = npurenoise // 2
        nflat = (npurenoise - ngaus) // 2
        nexpo = npurenoise - nflat - ngaus
        noise_gaus = np.random.normal( 0, 1, (amount//2+amount, ngaus))
        noise_flat = np.random.uniform(-1, 1, (amount//2+amount, nflat))
        noise_expo = np.random.exponential( 1,    (amount//2+amount, nexpo))
        truenoise = X_train[np.random.randint(0, X_train.shape[0], amount//2+amount)]
        noise = np.c_[ truenoise,noise_gaus,noise_flat,noise_expo ]
        noise_half = noise[:amount//2]
        noise_full = noise[-amount:]

    return X_train, noise_half, noise_full



In [94]:
def covariance_metrics(real_data, predictions):
    """Takes in real_data matrix with real entries as rows and predictions matrix with generated events as rows and returns the covariance matricies for the two as well as the average, maximum, and std. dev of the difference between the entries in the coverance matrix as well as in the average of the variables."""
    
    cov_pred = np.cov(predictions.T)
    avg_pred = predictions.mean(axis=0)
    cov_real = np.cov(real_data.T)
    avg_real = real_data.mean(axis=0)
    
    cov_diff = (cov_pred - cov_real)
    avg_diff = (avg_pred - avg_real)/(np.sqrt(np.abs(avg_pred*avg_real)))
    
    return cov_pred, cov_real, cov_diff.mean(), cov_diff.max(), cov_diff.std(), avg_diff.mean(), avg_diff.max(), avg_diff.std()

    

In [103]:
# get noise, predict from it, and plot stuff. easy.

# You must load the config of the model you want into the gan first by running the block of code with the
# proper config settings or the loss function will be messed up here.

tag = "v2_512_bgbd_mllANDwidth_NonTC"

#Old file with Gen/Reco mix
input_file = "/home/users/bhashemi/Projects/GIT/DY-GAN/delphes/data_Nov10.npa"
#New Files with Gen and Reco split
#input_file = "/home/users/bhashemi/Projects/GIT/DY-GAN/delphes/total_Zmumu_13TeV_PU20_v2.npa"

data = load_data(input_file)
#loss_type = "force_mll",
loss_type = "force_z_width"
loss_weight = 0.01

#noise_shape = (8,)
noise_shape = (19,)

cov_avg = {}
cov_std_dev = {}
cov_max = {}
mean_avg = {}
mean_std_dev = {}
mean_max = {}

for i in range(500,1000000,500):
    epoch=i
    try:
        model = load_model("progress/%s/gen_%d.weights" % (tag,epoch), custom_objects={'loss_func': custom_loss(c=loss_weight, loss_type=loss_type)})
    except Exception as e:
        continue
    real_events, noise_half, noise = get_noise(input_file, data, 1, noise_shape, 50000)
    real = real_events[:50000]

    preds = model.predict(noise)
    #print (preds-real)
    #print (preds-real).mean(axis=0)
    cov_pred, cov_real, avg_diff, max_diff, std_dev_diff, mean_avg_diff, max_avg_diff, std_dev_avg_diff = covariance_metrics(real, preds)
    
    cov_avg[i] = avg_diff
    cov_std_dev[i] = std_dev_diff
    cov_max[i] = max_diff
    mean_avg[i] = mean_avg_diff
    mean_std_dev[i] = std_dev_avg_diff
    mean_max[i] = max_avg_diff
    print("epoch %d: %f %f %f %f %f %f" % (i, avg_diff, max_diff, std_dev_diff, mean_avg_diff, max_avg_diff, std_dev_avg_diff) )

print(cov_avg)
print("=====================")
print(cov_std_dev)
print("=====================")
print(cov_max)
print("=====================")
print(mean_avg)
print("=====================")
print(mean_std_dev)
print("=====================")
print(mean_max)
print("=====================")

print(sorted(cov_max.items(), key=lambda(k,v):v))
    
"""epoch=52000
#loss_type = "force_mll",
loss_type = "force_z_width"
loss_weight = 0.01

#noise_shape = (8,)
noise_shape = (19,)

print("progress/%s/gen_%d.weights" % (tag,epoch))
model = load_model("progress/%s/gen_%d.weights" % (tag,epoch), custom_objects={'loss_func': custom_loss(c=loss_weight, loss_type=loss_type)})
real_events, noise_half, noise = get_noise(input_file, data, 1, noise_shape, 50000)
real = real_events[:50000]

preds = model.predict(noise)
#print (preds-real)
#print (preds-real).mean(axis=0)
cov_pred, cov_real, avg_diff, max_diff, std_dev_diff, mean_avg_diff, max_avg_diff, std_dev_avg_diff = covariance_metrics(real, preds)

print(avg_diff, max_diff, std_dev_diff, mean_avg_diff, max_avg_diff, std_dev_avg_diff)

#print(cov_real[2,2], cov_pred[2,2], cov_diff[2,2])
# make_plots(noise,gan.data[:5000],title="epoch {}".format(3000))]
make_plots(preds,data[:50000],title="epoch {}".format(epoch), visible=True)"""

epoch 500: -186.691876 780.755211 1339.332606 5.402335 41.059963 18.561598
epoch 1000: -89.319955 4482.799234 1120.521566 -3.041015 32.136623 19.451509
epoch 1500: 10.705658 4939.761791 1119.040668 5.903584 41.758865 13.827611
epoch 2000: -194.652825 232.203983 1355.255473 -7.057598 35.289108 18.617411
epoch 2500: -194.729748 707.729980 1336.978276 -10.678638 7.750679 19.397617
epoch 3000: -115.646613 1847.321038 1084.826462 0.230484 23.027134 14.770062
epoch 3500: -108.482689 2440.022011 999.699213 3.190890 34.986996 11.685556
epoch 4000: -114.370362 2818.653884 982.500747 0.081487 36.849300 13.998690
epoch 4500: -108.418956 3662.854749 947.027994 4.996789 36.509800 10.357778
epoch 5000: -190.659380 88.566482 1370.969061 -2.015435 6.664252 5.350131
epoch 5500: -185.400901 306.432144 1231.655609 -1.054888 42.510654 15.264757
epoch 6000: -155.720984 215.981822 868.832205 -0.354947 12.720071 5.878477
epoch 6500: -175.690280 399.806190 1117.834866 -1.620004 26.184799 11.828341
epoch 7000:

epoch 59500: -165.129761 433.880801 860.012303 -1.738160 7.717324 6.634615
epoch 60000: -60.977904 374.185048 346.741862 -2.034546 4.271807 6.114116
epoch 60500: -103.957773 313.813266 715.064752 -1.943597 2.847266 3.271551
epoch 61000: -95.718432 622.553087 577.287781 -0.846360 5.088030 3.051794
epoch 61500: -70.766243 3213.668292 723.192856 -1.029684 10.472175 12.770995
epoch 62000: -150.774046 242.690204 782.310204 -1.144841 5.787592 4.831378
epoch 62500: -59.966349 1148.942004 381.306987 -0.677809 5.864537 3.297674
epoch 63000: -81.691626 158.382555 424.407191 -0.413406 5.667912 3.116580
epoch 63500: -92.611021 64.853259 462.083507 -1.027548 5.614676 3.092683
epoch 64000: -160.636923 190.495105 1089.565819 -1.651120 5.760307 2.881233
epoch 64500: -145.720920 119.996584 736.477851 -2.578414 3.868884 5.217415
epoch 65000: -92.711884 137.892338 482.209794 -0.829381 6.857785 4.368003
epoch 65500: -124.145153 239.200827 814.750697 -1.113763 3.801489 4.609635
epoch 66000: -115.939084 502

IOError: Unable to open file (unable to open file: name = 'progress/v2_512_bgbd_mllANDwidth_NonTC/gen_100500.weights', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)